In [50]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable


In [51]:
!pip install selenium

Defaulting to user installation because normal site-packages is not writeable


In [52]:
!pip install bs4

Defaulting to user installation because normal site-packages is not writeable


In [74]:
#!apt-get update

In [75]:
#!apt-get install chromium chromium-driver

In [76]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import time


def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver




In [77]:
driver = web_driver()




In [78]:
driver.get('https://www.amazon.in/')

In [79]:
driver.maximize_window()

In [80]:
time.sleep(5)
isearch = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.NAME,'field-keywords')))
sbutton = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.ID, 'nav-search-submit-button')))

time.sleep(5)
isearch = driver.find_element(By.NAME,'field-keywords')
sbutton = driver.find_element(By.ID, 'nav-search-submit-button')

In [81]:
isearch.send_keys("iPhone")
time.sleep(1)

In [82]:
sbutton.click()

In [83]:
from bs4 import BeautifulSoup
html = driver.page_source

soup = BeautifulSoup(html)


In [84]:
#button_element = soup.find('a', {'data-cel-widget':'search_result_2'})


In [85]:
button_element = soup.find('a', {'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})


In [86]:
print(button_element)

<a class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal" href="/Apple-iPhone-13-128GB-Blue/dp/B09G9BL5CP/ref=sr_1_1?crid=ZXCSN0YAG61M&amp;dib=eyJ2IjoiMSJ9.eFa-TvbcC_zjCq_5PD2KO7sua8FLjHkgudzmYiS71BIYxc61NR9Qh-S3XchUD4-C3CMBdzG8ftNvyDkklDuV1S9UDQCQYNvS9gDHQ02CT6DMnEaXSLZpjcel1vH-ClWKRmcQAtSSDHdq2L6T0MUwZ84DbR8HmC4Fp99AIO78fEnshYLaBRD3dOHU5M84IjgwT7u2jjtupnLZMF1lIEiqIjByhuDc83JnaEQ8TnZYdvY.VWPcnNjJgZ6TQ_0WxVsLqygEy_TKZM6vv3urrJkGmsc&amp;dib_tag=se&amp;keywords=iPhone&amp;qid=1716392621&amp;sprefix=iphone%2Caps%2C350&amp;sr=8-1" target="_blank"><span class="a-size-medium a-color-base a-text-normal">Apple iPhone 13 (128GB) - Blue</span> </a>


In [87]:
button_class = button_element['class']

In [88]:
print(button_class)

['a-link-normal', 's-underline-text', 's-underline-link-text', 's-link-style', 'a-text-normal']


from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By


In [89]:
# Construct the CSS selector using the button class
button_selector = f".{'.'.join(button_class)}"

# Find the element using Selenium
element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, button_selector)))


In [90]:
# Perform actions (e.g., click)
element.click()

In [91]:
driver.switch_to.window(driver.window_handles[1])

In [92]:

time.sleep(10)

In [93]:
morereviewbutton = "See more reviews"
morereviewbutton1 = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.LINK_TEXT, morereviewbutton)))
morereviewbutton1.click()

In [94]:
url1 = driver.current_url
print(url1)

https://www.amazon.in/Apple-iPhone-13-128GB-Blue/product-reviews/B09G9BL5CP/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews


In [95]:
html = driver.page_source

soup = BeautifulSoup(html)


In [96]:
import time
import pandas as pd

# Initialize an empty DataFrame
df = pd.DataFrame(columns=['title', 'rating', 'date', 'body', 'details'])

# Loop through the desired number of pages
for x in range(1, 101):
    time.sleep(2)
    url = f"{url1}&pageNumber={x}"
    driver.get(url)
    html = driver.page_source
    #soup = get_soup(url)
    reviews = soup.find_all('div', {'data-hook': 'review'})
    
    # Initialize empty lists to store data for this page
    titles = []
    ratings = []
    dates = []
    bodies = []
    details = []

    # Extract information for each review on this page
    for item in reviews:
        # Extract information from the current review item
        title = item.find('a', {'data-hook': 'review-title'}).text.strip() if item.find('a', {'data-hook': 'review-title'}) else pd.NA
        titles.append(title)
        
        rating = item.find('i', {'data-hook': 'review-star-rating'}).text if item.find('i', {'data-hook': 'review-star-rating'}) else pd.NA
        ratings.append(rating)
        
        date = item.find('span', {'data-hook': 'review-date'}).text if item.find('span', {'data-hook': 'review-date'}) else pd.NA
        dates.append(date)
        
        body = item.find('span', {'data-hook': 'review-body'}).text if item.find('span', {'data-hook': 'review-body'}) else pd.NA
        bodies.append(body)
        
        detail = item.find('a', {'data-hook': 'format-strip'}).text if item.find('a', {'data-hook': 'format-strip'}) else pd.NA
        details.append(detail)

    # Append the data for this page to the DataFrame
    page_df = pd.DataFrame({'title': titles, 'rating': ratings, 'date': dates, 'body': bodies, 'details': details})
    df = pd.concat([df, page_df], ignore_index=True)
    
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")


    # Wait for the next page to load
    time.sleep(5)

    print("Scraped Page No.", x)


# Reset index
df.reset_index(drop=True, inplace=True)


Scraped Page No. 1
Scraped Page No. 2
Scraped Page No. 3
Scraped Page No. 4
Scraped Page No. 5
Scraped Page No. 6
Scraped Page No. 7
Scraped Page No. 8
Scraped Page No. 9
Scraped Page No. 10
Scraped Page No. 11
Scraped Page No. 12
Scraped Page No. 13
Scraped Page No. 14
Scraped Page No. 15
Scraped Page No. 16
Scraped Page No. 17
Scraped Page No. 18
Scraped Page No. 19
Scraped Page No. 20
Scraped Page No. 21
Scraped Page No. 22
Scraped Page No. 23
Scraped Page No. 24
Scraped Page No. 25
Scraped Page No. 26
Scraped Page No. 27
Scraped Page No. 28
Scraped Page No. 29
Scraped Page No. 30
Scraped Page No. 31
Scraped Page No. 32
Scraped Page No. 33
Scraped Page No. 34
Scraped Page No. 35
Scraped Page No. 36
Scraped Page No. 37
Scraped Page No. 38
Scraped Page No. 39
Scraped Page No. 40
Scraped Page No. 41
Scraped Page No. 42
Scraped Page No. 43
Scraped Page No. 44
Scraped Page No. 45
Scraped Page No. 46
Scraped Page No. 47
Scraped Page No. 48
Scraped Page No. 49
Scraped Page No. 50
Scraped P

In [97]:
df.head(50)

,title,rating,date,body,details
0,5.0 out of 5 stars\nRedefining Excellence: A R...,5.0 out of 5 stars,Reviewed in India on 24 February 2024,\nThe iPhone 13 128GB has surpassed my expecta...,<NA>
1,5.0 out of 5 stars\nUsing after 3 months hones...,5.0 out of 5 stars,Reviewed in India on 30 December 2023,\n\n\n\n\n The media could ...,Colour: BlueSize: 128 GB
2,4.0 out of 5 stars\nThe iPhone 13 - A Stunning...,4.0 out of 5 stars,Reviewed in India on 1 November 2023,"\nThe iPhone 13 is, without a doubt, a game-ch...",Colour: StarlightSize: 256 GB
3,"5.0 out of 5 stars\nSo Beautiful, So elegant, ...",5.0 out of 5 stars,Reviewed in India on 13 November 2023,\nI snagged the iPhone 13 during the Great Ind...,Colour: MidnightSize: 128 GB
4,5.0 out of 5 stars\nA gift for my wife,5.0 out of 5 stars,Reviewed in India on 21 April 2024,\nStep into a world of sophistication with the...,<NA>
5,5.0 out of 5 stars\nAbsolute brilliance!,5.0 out of 5 stars,Reviewed in India on 9 May 2024,\nYou don't need to explain how good apple pro...,Colour: BlueSize: 128 GB
6,5.0 out of 5 stars\nDo I need to say something...,5.0 out of 5 stars,Reviewed in India on 30 March 2024,\nGreat product.. I love android and apple bot...,<NA>
7,5.0 out of 5 stars\nAwesome phone,5.0 out of 5 stars,Reviewed in India on 20 May 2024,\nValue for time and money also.Nice products\n,<NA>
8,4.0 out of 5 stars\nAndroid to Apple Transitio...,4.0 out of 5 stars,Reviewed in India on 4 February 2023,\nThis is my first iPhone and it always feels ...,Colour: BlueSize: 128 GB
9,4.0 out of 5 stars\nMobile have more heat at t...,4.0 out of 5 stars,Reviewed in India on 11 May 2024,\nAll over good price is good less than other ...,Colour: GreenSize: 128 GB


In [98]:
df.to_excel("FinalData.xlsx")